In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/Plant_Village',shuffle = True,
                                                              image_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size = BATCH_SIZE)

Found 2152 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names

In [5]:
len(dataset) #you get 68 because it is taking batch of 32

68

In [6]:
def get_dataset_partition(ds,train_split = 0.8, test_split = 0.1, val_split = 0.1 , shuffle = True, shuffle_size = 10000):
  ds_size = len(ds)
  if shuffle:
    ds.shuffle(shuffle_size, seed = 12)

  train_size = int(train_split*ds_size)
  test_size = int(test_split*ds_size)

  train_ds = ds.take(train_size)
  val_ds = ds.skip(train_size).take(test_size)
  test_ds = ds.skip(train_size).skip(test_size)

  return train_ds, val_ds, test_ds


In [7]:
train_ds, val_ds, test_ds = get_dataset_partition(dataset)

In [8]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [9]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

In [10]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [11]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [12]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [13]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=10,
)

Epoch 1/10
54/54 [==============================] - 503s 5s/step - loss: 0.8761 - accuracy: 0.5359 - val_loss: 0.7634 - val_accuracy: 0.6510
Epoch 2/10
54/54 [==============================] - 243s 5s/step - loss: 0.5983 - accuracy: 0.7303 - val_loss: 0.6182 - val_accuracy: 0.7656
Epoch 3/10
54/54 [==============================] - 257s 5s/step - loss: 0.4347 - accuracy: 0.8137 - val_loss: 0.4851 - val_accuracy: 0.8073
Epoch 4/10
54/54 [==============================] - 250s 5s/step - loss: 0.3306 - accuracy: 0.8675 - val_loss: 0.2842 - val_accuracy: 0.8854
Epoch 5/10
54/54 [==============================] - 253s 5s/step - loss: 0.3019 - accuracy: 0.8692 - val_loss: 0.2941 - val_accuracy: 0.8958
Epoch 6/10
54/54 [==============================] - 249s 5s/step - loss: 0.2000 - accuracy: 0.9167 - val_loss: 0.1825 - val_accuracy: 0.9115
Epoch 7/10
54/54 [==============================] - 242s 4s/step - loss: 0.2172 - accuracy: 0.9120 - val_loss: 0.1542 - val_accuracy: 0.9219
Epoch 8/10
54

In [14]:
scores = model.evaluate(test_ds)

8/8 [==============================] - 14s 985ms/step - loss: 0.1845 - accuracy: 0.9267
